<a href="https://colab.research.google.com/github/Chenzhang0329/Credit_Score_Classification/blob/main/6100_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


This study is using a banking dataset posted on Kaggle, and it has a train and a test dataset.
It contains 100,000 customers, with 27 different variables that hold the customer information. In general, the dataset contains numerical, binary information, and categorical variables, and this information includes: Customer ID, Month, Name, Age, SSN(social security number), Occupation, Annual
Income, Monthly Inhand Salary, Num Bank Accounts, Num Credit Card, Interest Rate, Num
of Loan, Type of Loan, Delay from due date(average number of days delayed from the payment
date), Num of Delayed Payment(average number of payments delayed by a person), Changed Credit
Limit(percentage change in credit card limit), Num Credit Inquiries, Credit Mix(classification of
the mix of credits), Outstanding Debt, Credit Utilization Ratio, Credit History Age:(age of credit
history of the person), Payment of Min Amount, Total EMI per month(total Equated Monthly
Installments payments (in USD)), Amount invested monthly, Payment Behaviour, Monthly Balance
and Credit Score(Poor, Standard, Good).

Based on these information, the financial institutions would like to make their credit report and determine an individual's credit score to make further decisions such as granting loans to the person or deciding the interest rate on the loans. Our goal will be to use these information to build machine
learning models that can classify the credit scores of the customers.

# Design

For our final project, we are going to use different models to approach the classification, which are Random Forest, K-nn, and Support Vector Machines. We will compute the precision, recall, and accuracy for comparision between the models.

# Ethics

This dataset includes many of the clients' personal credit-related details, such as their names, homes, annual income, and so on. The release of personal information should require clients' consent, assuming they are real. Without their consent (of using their data for this task), the bank should not have released this dataset to us, as there are huge privacy concerns that may affect the individuals.

# Importing libraries and data

In [ ]:
###
# Upload and unzip data ("CreditScore.zip")
###

from google.colab import files
from zipfile import ZipFile

uploaded = files.upload()
with ZipFile('CreditScore.zip') as z:
    z.extractall()

Saving CreditScore.zip to CreditScore (1).zip


In [ ]:
# importing libraries
import math
import pandas as pd
import numpy as np

# Globally set the printing option 
pd.set_option('display.max_columns', None)

In [ ]:
# importing data
train_data = pd.read_csv('/content/CreditScore/train.csv')
test_data = pd.read_csv('/content/CreditScore/test.csv')

train_data.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


# Data Cleaning

Data cleaning is done before any models are fit to the dataset. Kaggle provides the descriptions for this dataset, and based on the descriptions the following actions are done:


*   Converted variables into numbers if they fall into the following categories:


>> Originally was string (eg. Age, Annual_Income)

>> The values resemble some ordering (eg. Month)


*   Separated Type_of_Loan and Payment_Behaviour into different categories
*   One-hot encode dummy variables (drop_first=True) for Occupation
*   NA: filled to 0s or values that already exist in the data

>> If a person's annual income was available for all but one entry, use the previous entries

>> Some values that don't make sense (e.g. -500 for age) are also treated as NAs and dealt accordingly


*   Customer_ID, Name, SSN, Type_of_Loan, Payment_Behaviour are removed for training purposes

In [ ]:
#### Modify train and test data

# Transported is the value we want to predict
Credit_Score = train_data['Credit_Score']

# for data transformation

# Combine train and test
temp = [train_data, test_data]
# Remove Transported from train data to combine with test data
full_data = pd.concat(temp, keys=['train', 'test']).drop('Credit_Score', axis=1)

## Convert into numerical values

In [ ]:
# string to number (function)
def str_to_num(data):
  return pd.to_numeric(data.apply(lambda x: x.replace('_', '') if type(x) == str and '_' in x else x))

def new_column(data, prim_key, to_change, min_bound, max_bound):
  valid_vals = {}
  new_col = []
  #data[to_change] = pd.to_numeric(data[to_change])

  for x,y in zip(data[prim_key], data[to_change]):
    if x not in valid_vals:
      valid_vals[x] = []
    if not pd.isnull(y):
      if y not in valid_vals[x] and y >= min_bound and y <= max_bound:
        valid_vals[x].append(y)
  
  for x,y in zip(data[prim_key], data[to_change]):
    if y in valid_vals[x]:
      new_col.append(y)
    else:
      new_col.append(valid_vals[x][0])
  
  return valid_vals, new_col

In [ ]:
### string to number
#TODO: DELETE LATER, FOR TESTING
full_data = pd.concat(temp, keys=['train', 'test']).drop('Credit_Score', axis=1)
# Age
full_data['Age'] = str_to_num(full_data['Age'])
_, full_data['Age'] = new_column(full_data, 'Customer_ID', 'Age', 0, 150)

# Annual_Income
full_data['Annual_Income'] = str_to_num(full_data['Annual_Income'])

# Num_of_Loan
#TODO: change after Type_of_Loan is converted

# Num_of_Delayed_Payment
full_data['Num_of_Delayed_Payment'] = str_to_num(full_data['Num_of_Delayed_Payment'])
_, full_data['Num_of_Delayed_Payment'] = new_column(full_data, 'Customer_ID', 'Num_of_Delayed_Payment', 
                                                    full_data['Num_of_Delayed_Payment'].min(), 
                                                    full_data['Num_of_Delayed_Payment'].max())

# Changed_Credit_Limit
full_data['Changed_Credit_Limit'] = str_to_num(full_data['Changed_Credit_Limit'])
_, full_data['Changed_Credit_Limit'] = new_column(full_data, 'Customer_ID', 'Num_of_Delayed_Payment', 
                                                   full_data['Changed_Credit_Limit'].min(), 
                                                   full_data['Changed_Credit_Limit'].max())

# Outstanding_Debt
full_data['Outstanding_Debt'] = str_to_num(full_data['Outstanding_Debt'])

# Credit_History_Age
# TODO

# Amount_invested_monthly
full_data['Amount_invested_monthly'] = full_data['Amount_invested_monthly'].apply(lambda x: "0" if x == "__10000__" else x)
full_data['Amount_invested_monthly'] = str_to_num(full_data['Amount_invested_monthly'])
full_data['Amount_invested_monthly'].fillna(0, inplace=True)

In [ ]:
# rankings to numerical
convert_dict = {'Month': {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
                          'May': 5, 'June': 6, 'July': 7, 'August': 8, 
                          'September': 9, 'October': 10, 'November': 11, 'December': 12},
                'Credit_Mix': {'Bad': -1, 'Standard': 0, 'Good': 1, '_':0},
                'Payment_of_Min_Amount': {'No': 0, 'NM': 0, 'Yes': 1}}

for var, conversion in convert_dict.items():
  for old_val, new_val in conversion.items():
    full_data.loc[full_data[var] == old_val, var] = new_val

## Create new variables from Type_of_Loan and Payment_Behaviour

### Type_of_Loan

There are some missing values in "Type_of_Loan", and we want to fill those missing values first. But for "Type_of_loan", those missing values means "No Loan" instead of "Not specified". So we will created a new type "No Loan" to fill those missing values.

In [ ]:
## Filling all the missing value for "Type_of_Loan" as "No Loan"
full_data["Type_of_Loan"] = full_data["Type_of_Loan"].fillna("No Loan")

After filling all the missing values for "Type_of_Loan", we noticed that "Num_of_Loan" has some values that don't make much sense. Some of them have 967 loans, and some of them have -100 loans. We will fix this column based on the "Type_of_Loan".

In [ ]:
# Fix the "Num_of_Loan" based on the "Type_of_Loan"
# TODO: change to count of all loans
full_data['Num_of_Loan'] = full_data.Type_of_Loan.str.split(', ').map(lambda x: len(x))
full_data.loc[full_data.Type_of_Loan == "No Loan",'Num_of_Loan'] = 0
full_data.loc[full_data.Type_of_Loan == "Not Specified",'Num_of_Loan'] = 1

We can see there are only 10 different types of loans in the dataset, so we will create 10 new bernoulli variables with 0 and 1 based on the "Type_of_Loan".

In [ ]:
# Create a list for all the Type of Loan
full_data['Type_of_Loan'] = full_data.Type_of_Loan.str.replace(" ", "").str.replace("and", "")
Loan_type_lst = []
for item in list(full_data.Type_of_Loan.str.split(',')):
  for element in item:
    if element not in Loan_type_lst:
      Loan_type_lst.append(element)

print(Loan_type_lst)


['AutoLoan', 'Credit-BuilderLoan', 'PersonalLoan', 'HomeEquityLoan', 'NotSpecified', 'NoLoan', 'MortgageLoan', 'StudentLoan', 'DebtConsolidationLoan', 'PaydayLoan']


In [ ]:
for item in Loan_type_lst:
  full_data[item] = full_data.Type_of_Loan.str.replace(" ", "").replace("and", "").str.split(',').map(lambda x: 1 if item in x else 0)

### Payment_Behaviour

In [ ]:
full_data['Payment_Behaviour'] = full_data['Payment_Behaviour'].replace('!@9#%8', "None")

full_data['Spend_Behaviour'] = full_data.Payment_Behaviour.str.split('_').map(lambda x: x[0] if len(x) > 1 else "None")
full_data['Value_Behaviour'] = full_data.Payment_Behaviour.str.split('_').map(lambda x: x[2] if len(x) > 1 else "None")

In [ ]:
full_data.head()

ID Customer_ID Month           Name  Age          SSN Occupation  \
train 0  0x1602   CUS_0xd40     1  Aaron Maashoh   23  821-00-0265  Scientist   
      1  0x1603   CUS_0xd40     2  Aaron Maashoh   23  821-00-0265  Scientist   
      2  0x1604   CUS_0xd40     3  Aaron Maashoh   23  821-00-0265  Scientist   
      3  0x1605   CUS_0xd40     4  Aaron Maashoh   23  821-00-0265  Scientist   
      4  0x1606   CUS_0xd40     5  Aaron Maashoh   23  821-00-0265  Scientist   

         Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
train 0       19114.12            1824.843333                  3   
      1       19114.12                    NaN                  3   
      2       19114.12                    NaN                  3   
      3       19114.12                    NaN                  3   
      4       19114.12            1824.843333                  3   

         Num_Credit_Card  Interest_Rate  Num_of_Loan  \
train 0                4              3            4   
      1                4              3            4   
      2                4              3            4   
      3                4              3            4   
      4                4              3            4   

                                              Type_of_Loan  \
train 0  AutoLoan,Credit-BuilderLoan,PersonalLoan,HomeE...   
      1  AutoLoan,Credit-BuilderLoan,PersonalLoan,HomeE...   
      2  AutoLoan,Credit-BuilderLoan,PersonalLoan,HomeE...   
      3  AutoLoan,Credit-BuilderLoan,PersonalLoan,HomeE...   
      4  AutoLoan,Credit-BuilderLoan,PersonalLoan,HomeE...   

         Delay_from_due_date  Num_of_Delayed_Payment  Changed_Credit_Limit  \
train 0                    3                     7.0                   7.0   
      1                   -1                     7.0                   7.0   
      2                    3                     7.0                   7.0   
      3                    5                     4.0                   4.0   
      4                    6                     7.0                   7.0   

         Num_Credit_Inquiries Credit_Mix  Outstanding_Debt  \
train 0                   4.0          0            809.98   
      1                   4.0          1            809.98   
      2                   4.0          1            809.98   
      3                   4.0          1            809.98   
      4                   4.0          1            809.98   

         Credit_Utilization_Ratio     Credit_History_Age  \
train 0                 26.822620  22 Years and 1 Months   
      1                 31.944960                    NaN   
      2                 28.609352  22 Years and 3 Months   
      3                 31.377862  22 Years and 4 Months   
      4                 24.797347  22 Years and 5 Months   

        Payment_of_Min_Amount  Total_EMI_per_month  Amount_invested_monthly  \
train 0                     0            49.574949                80.415295   
      1                     0            49.574949               118.280222   
      2                     0            49.574949                81.699521   
      3                     0            49.574949               199.458074   
      4                     0            49.574949                41.420153   

                        Payment_Behaviour     Monthly_Balance  AutoLoan  \
train 0   High_spent_Small_value_payments  312.49408867943663         1   
      1    Low_spent_Large_value_payments  284.62916249607184         1   
      2   Low_spent_Medium_value_payments   331.2098628537912         1   
      3    Low_spent_Small_value_payments  223.45130972736786         1   
      4  High_spent_Medium_value_payments  341.48923103222177         1   

         Credit-BuilderLoan  PersonalLoan  HomeEquityLoan  NotSpecified  \
train 0                   1             1               1             0   
      1                   1             1               1             0   
      2                   1             1            

# Fixed the 'Occupation' column

In [ ]:
df = full_data[full_data['Occupation'] != '_______']
occup_dict = pd.Series(full_data.Occupation.values,index=full_data.Customer_ID).to_dict()

In [ ]:
print(occup_dict)

{'CUS_0xd40': 'Scientist', 'CUS_0x21b1': 'Teacher', 'CUS_0x2dbc': 'Engineer', 'CUS_0xb891': 'Entrepreneur', 'CUS_0x1cdb': 'Developer', 'CUS_0x95ee': 'Lawyer', 'CUS_0x284a': 'Lawyer', 'CUS_0x5407': 'Media_Manager', 'CUS_0x4157': 'Doctor', 'CUS_0xba08': 'Journalist', 'CUS_0xa66b': 'Teacher', 'CUS_0xc0ab': 'Engineer', 'CUS_0x3e45': 'Entrepreneur', 'CUS_0x6c66': 'Manager', 'CUS_0xff4': 'Entrepreneur', 'CUS_0x33d2': '_______', 'CUS_0x6070': 'Accountant', 'CUS_0xfdb': 'Teacher', 'CUS_0x3553': 'Musician', 'CUS_0x4100': 'Entrepreneur', 'CUS_0x132f': 'Musician', 'CUS_0xa16e': 'Media_Manager', 'CUS_0xac86': '_______', 'CUS_0x5b48': 'Lawyer', 'CUS_0xa5f9': 'Mechanic', 'CUS_0x4d43': 'Entrepreneur', 'CUS_0xb95f': 'Writer', 'CUS_0x3edc': 'Accountant', 'CUS_0xbffe': 'Architect', 'CUS_0xb681': 'Manager', 'CUS_0x9a71': 'Scientist', 'CUS_0x6048': 'Musician', 'CUS_0x95b5': 'Engineer', 'CUS_0x4004': 'Writer', 'CUS_0xb101': '_______', 'CUS_0x9b94': 'Manager', 'CUS_0x4080': 'Mechanic', 'CUS_0x706a': 'Engine

## One hot encode dummy variables for categorical variables

In [ ]:
#one hot encode
categorical_var = ['Occupation']

full_data = pd.get_dummies(full_data, columns=categorical_var, drop_first=True)

## Delete unneeded variables

In [ ]:
# delete columns that are unneeded
# TODO: add the comment below after all other data cleaning are done
to_delete = [] #['Customer_ID', 'Name', 'SSN', 'Type_of_Loan', 'Payment_Behaviour']
full_data.drop(columns=to_delete, inplace=True)


## Split back to train and test set

In [ ]:
# split back to train and test
train_data = full_data.loc['train']
test_data = full_data.loc['test'] # this is our final test set for our results

# drop ids in training data as it's not needed
train_data = train_data.drop(columns=['ID'])
test_ids = test_data['ID'] # keep for final prediction
test_data = test_data.drop(columns=['ID'])

# for training
# x_train = train_data
# y_train = Transported